<a href="https://colab.research.google.com/github/js0807/ai_study/blob/main/FlaskPage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flask 기초

In [2]:
from flask import Flask
from flask_ngrok import run_with_ngrok

app=Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
  return "<h1>This is your Flask server.</h1>"

@app.route("/predict")
def predict():
  message=""
  message+="<h1>House Prices</h1>"
  message+="This page will be your prediction form."

  return message

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://edddf4ecf6ef.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [09/Aug/2021 12:06:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2021 12:06:45] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Aug/2021 12:06:50] "GET /predict HTTP/1.1" 200 -


# 실전

## Import

In [22]:
!pip install flask-ngrok

In [23]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.preprocessing import MinMaxScaler

## Preprocess data

In [40]:
cd /content/drive/MyDrive/data/house_prices/

/content/drive/MyDrive/data/house_prices


In [25]:
# load data
X = pd.read_csv('X.csv')

with open('y.npy', 'rb') as f:
    y = np.load(f)

# OverallQual 10
# GrLivArea 10
# GarageCars 9
# GarageArea 8
# TotalBsmtSF 7
# 1stFlrSF 6
# FullBath 5
# LotShape Reg
X = X[['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'FullBath', 'LotShape_rank']]

x_min_max_scaler = MinMaxScaler()
x_min_max_scaler.fit(X)
scaled_X = x_min_max_scaler.transform(X)

y_min_max_scaler = MinMaxScaler()
y_min_max_scaler.fit(y)
scaled_y = y_min_max_scaler.transform(y)

## Training & Save Model

In [41]:
model = keras.Sequential(
      [
          keras.Input(shape=scaled_X.shape[-1]),
          layers.Dense(96, activation='relu'),
          layers.Dense(48, activation='relu'),
          layers.Dense(1)
      ]
  )

model.compile(loss="mse", optimizer="adam")

early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)
model.fit(scaled_X, scaled_y, 
          batch_size=2, epochs=150, 
          callbacks=[early_stopping_callback], validation_split=0.05)

Epoch 1/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0040 - val_loss: 0.0018
Epoch 2/150
694/694 [==============================] - 1s 1000us/step - loss: 0.0034 - val_loss: 0.0032
Epoch 3/150
694/694 [==============================] - 1s 984us/step - loss: 0.0032 - val_loss: 0.0018
Epoch 4/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0027 - val_loss: 0.0016
Epoch 5/150
694/694 [==============================] - 1s 987us/step - loss: 0.0027 - val_loss: 0.0016
Epoch 6/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0026 - val_loss: 0.0037
Epoch 7/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0026 - val_loss: 0.0019
Epoch 8/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0027 - val_loss: 0.0013
Epoch 9/150
694/694 [==============================] - 1s 983us/step - loss: 0.0026 - val_loss: 0.0021
Epoch 10/150
694/694 [==============================] - 1s 986us/step - loss: 0.00

In [42]:
!pwd

/content/drive/My Drive/data/house_prices


In [43]:
model.save("mlp_v0.1.h5")

In [44]:
reconstructed_model = keras.models.load_model("mlp_v0.1.h5")

In [27]:
# with no saved model
# pred = model.predict(scaled_X[:5]) # 0 ~ 1
# pred = y_min_max_scaler.inverse_transform(pred)

# with saved model
pred = reconstructed_model.predict(scaled_X[:1]) # 0 ~ 1
pred = y_min_max_scaler.inverse_transform(pred)

## to Flask Page

In [45]:
template_folder_pwd='/content/drive/MyDrive/data/deployment/'

In [49]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok

# load data
X = pd.read_csv('X.csv')

with open('y.npy', 'rb') as f:
    y = np.load(f)

X = X[['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'FullBath', 'LotShape_rank']]

x_min_max_scaler = MinMaxScaler()
x_min_max_scaler.fit(X)

y_min_max_scaler = MinMaxScaler()
y_min_max_scaler.fit(y)

# load model
reconstructed_model = keras.models.load_model("mlp_v0.1.h5")

app=Flask(__name__,template_folder=template_folder_pwd)
run_with_ngrok(app)

def preprocess_data(data):
  X = [] # <-- OverallQual, GrLivArea, ... LotShape
  for k, v in data.items():
    if k == 'LotShape':
      if v == 'Reg':
        X.append(4)
      elif v == 'IR1':
        X.append(3)
      elif v == 'IR2':
        X.append(2)
      else:
        X.append(1)
    else:
      X.append(float(v))

  # X = [2, 5000, 2, ... , 3]
  X = np.array(X) # (8,)
  X = X.reshape((1, -1)) # (1, 8)

  # min max scaling
  scaled_X = x_min_max_scaler.transform(X)
  print(scaled_X)

  return scaled_X

@app.route("/")
def predict():
  return render_template('submit_form.html')

@app.route("/result",methods=['POST'])
def result():

  data=request.form

  message=""
  message+="<h1>House Prices</h1>"

  for k,v in data.items():
    message+=k+": "+v+"</br>"

  X=preprocess_data(data)

  pred = reconstructed_model.predict(X)
  pred = y_min_max_scaler.inverse_transform(pred)

  message+="<br><hr><br>"
  message+="Your House SalePrice is..."
  message+="<br>"
  message+="<h1 style='color:ForestGreen;'>$ "+str(pred[0][0])+" $</h1>"

  return message

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4f0fb1d7e976.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [09/Aug/2021 13:32:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2021 13:32:00] "GET /favicon.ico HTTP/1.1" 404 -


[[0.55555556 0.21288621 0.5        0.33850494 0.16219313 0.17278568
  0.66666667 0.66666667]]


...

[[0.55555556 0.21288621 0.5        0.33850494 0.16219313 0.17278568
  0.         0.        ]]


127.0.0.1 - - [09/Aug/2021 13:32:17] "POST /result HTTP/1.1" 200 -


[[0.55555556 0.21288621 0.5        0.33850494 0.16219313 0.17278568
  0.66666667 0.        ]]


127.0.0.1 - - [09/Aug/2021 13:32:22] "POST /result HTTP/1.1" 200 -


[[0.55555556 0.21288621 0.5        0.33850494 0.16219313 0.17278568
  1.66666667 0.        ]]


# +α
* deployment 진행 시 꼭! import 해야하는 package들의 version을 알아두고, 그거에 맞게 다운로드 진행하자!
    * !pip list | grep tensorflow, pandas, keras, ...
    * 이 것들을 편하게 설치하기 위해 requirements.txt에 각 패키지 및 버전 저장
        * ex) tensorflow==2.5.0
    * 그 후 pip install -r requirements.txt
* 서버에서 계속 돌려야 하는 경우. 백그라운드로 서버를 계속 돌려야한다.
    * nohub python3 flask-app.py &
    * 그러면 nohup.out 이라고 로그 남음
    * tail -f nohup.out # 계속 로그보기
